<a href="https://colab.research.google.com/github/JuanFelipePosada/python-basico_/blob/main/Dashboard_parcial_3_SYS_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit -q
!pip install pyngrok
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install soundfile
!pip install yt-dlp
!pip install yt-dlp soundfile matplotlib streamlit numpy
!pip install control


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.3 MB/s eta 0:00:00
     - 2.7 MB 9.6 MB/s 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata

In [5]:
%%writefile app.py
import requests
from io import BytesIO
import subprocess
import os
import shutil
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import tempfile
from scipy.fftpack import fft
from scipy.signal import square
from scipy.interpolate import interp1d
import yt_dlp
from scipy.signal import butter, lfilter, freqz, tf2zpk
import sympy as sp
from scipy import signal
import control as ctl

# Configuración inicial de la página
st.set_page_config(page_title="Parcial 2 - Fourier SyS", page_icon="📊", layout="wide")

# Barra lateral para seleccionar el punto del parcial y mostrar información general
with st.sidebar:
    st.image("https://lh4.googleusercontent.com/proxy/WNtyuTbDjnnITJFxg1dlI63L0jfIMRf0CIKg75VavFd3ameUuokpEiXIZvafO0UbA3rGKkhjDZ2HFtRWcGiPIn7Syd37PqnCrQuXFNHguRRPYm__safRJi9Q", width=150)
    st.title("Parcial 2 - Fourier SyS")
    st.subheader("Universidad Nacional de Colombia - Sede Manizales")
    st.write("**Señales y Sistemas - 2024-1**")
    st.write("**Integrantes:**")
    st.write("- Juan Felipe Posada Niasa")
    st.write("- Juan Jose Montilla Bastidas")

    # Menú desplegable para seleccionar el punto del parcial
    punto = st.selectbox("Selecciona el punto del parcial",
                         ("Punto 1: Modulación Sistema",
                          "Punto 2: Simulación Canción YouTube"))

# Mostrar información general del punto seleccionado
if punto == "Punto 1: Modulación Sistema":
  st.write("###  Modulación Sistema")
  # Definir variables simbólicas
  s = sp.symbols('s')
  m = sp.symbols('m')  # Masa como variable
  c_value = 1.0  # Valor fijo para el coeficiente de amortiguamiento
  k_value = 1.0  # Valor fijo para la constante del resorte
  wn, zeta = sp.symbols('omega_n zeta')  # Frecuencia natural y factor de amortiguamiento

  # Título de la aplicación
  st.title("Proceso para hallar la función de transferencia")

  # Descripción inicial
  st.write("""
  El objetivo es encontrar la función de transferencia del sistema masa-resorte-amortiguador.
  La ecuación de movimiento es:
  """)

  # Mostrar la ecuación diferencial
  st.latex(r"m \frac{d^2x(t)}{dt^2} + c \frac{dx(t)}{dt} + k x(t) = Y(t)")

  # Paso 1: Transformada de Laplace
  st.subheader("Paso 1: Transformada de Laplace")
  st.write("Aplicamos la transformada de Laplace a la ecuación diferencial, suponiendo condiciones iniciales cero:")
  st.latex(r"m s^2 X(s) + c s X(s) + k X(s) = Y(s)")

  # Paso 2: Despejar X(s)
  st.subheader("Paso 2: Despejar X(s)")
  st.write("Despejamos \( X(s) \) en términos de \( Y(s) \):")
  X_s = sp.symbols('X_s')
  Y_s = sp.symbols('Y_s')
  eq = sp.Eq(m * s**2 * X_s + c_value * s * X_s + k_value * X_s, Y_s)
  st.latex(sp.latex(eq))

  # Paso 3: Función de transferencia
  st.subheader("Paso 3: Función de transferencia")
  st.write("""
  La función de transferencia \( H(s) \) se define como la relación entre la salida \( X(s) \) y la entrada \( Y(s) \).
  Despejamos \( H(s) \):
  """)
  H_s = sp.simplify(Y_s / (m * s**2 + c_value * s + k_value))
  st.latex(r"H(s) = \frac{X(s)}{Y(s)} = \frac{1}{ms^2 + cs + k}")

  # Paso 4: Interpretación
  st.subheader("Paso 4: Interpretación")
  st.write("""
  Esta es la función de transferencia del sistema masa-resorte-amortiguador.
  Describe cómo la salida (el desplazamiento \( X(s) \)) responde a una entrada (la fuerza \( Y(s) \)).
  """)
  #
  # Mostrar la función de transferencia final
  st.write("La función de transferencia es:")
  st.latex(sp.latex(1 / (m * s**2 + c_value * s + k_value)))

  # Añadiendo la parte de los coeficientes y frecuencia natural
  st.subheader("Coeficientes y Reescritura en términos de \( omega_n ) y \( chi )")

  # Entradas para valores de m
  st.write("Ingrese el valor de la masa (m):")
  m_value = st.number_input("Masa (m)", value=1.0)

  # Calcular frecuencia natural wn y factor de amortiguamiento zeta
  wn_value = sp.sqrt(k_value / m_value)
  zeta_value = c_value / (2 * sp.sqrt(k_value * m_value))

  # Mostrar resultados de wn y zeta
  st.write("Resultados:")
  st.latex(r"\omega_n = \sqrt{\frac{k}{m}} = \sqrt{\frac{" + str(k_value) + "}{" + str(m_value) + "}} = " + str(wn_value.evalf()))
  st.latex(r"\zeta = \frac{c}{2\sqrt{km}} = \frac{" + str(c_value) + "}{2\sqrt{" + str(k_value) + " \cdot " + str(m_value) + "}} = " + str(zeta_value.evalf()))

  # Función de transferencia en términos de wn y chi
  st.write("""
  La función de transferencia también se puede escribir en términos de la frecuencia natural \( omega_n ) y el factor de amortiguamiento \( chi ):
  """)
  H_s = k_value * (wn**2 / (s**2 + 2 * zeta * wn * s + wn**2))
  st.latex(r"H(s) = k \frac{\omega_n^2}{s^2 + 2 \zeta \omega_n s + \omega_n^2}")

  st.write("Esta es la forma final en términos de \( omega_n ) y \( chi ).")
  ########################################################################################################################################################


  # Definir variables simbólicas para el modelado
  s = sp.symbols('s')
  wn, zeta = sp.symbols('omega_n zeta')  # Frecuencia natural y factor de amortiguamiento

  # Título del dashboard
  st.title("Simulación del Sistema Masa-Resorte-Amortiguador")

  # Descripción inicial
  st.write("""
  Este dashboard permite simular un sistema masa-resorte-amortiguador bajo diferentes condiciones de amortiguamiento.
  El usuario puede ajustar el valor del factor de amortiguamiento \( \zeta \) y visualizar la respuesta del sistema.
  """)

  # Entrada del factor de amortiguamiento
  zeta_value = st.slider('Seleccione el valor del factor de amortiguamiento (zeta)', 0.0, 2.0, 0.5, step=0.1)

  # Calcular parámetros del sistema en función de zeta y wn
  wn_value = 1.0  # Frecuencia natural fija
  c_value = 2 * zeta_value * wn_value
  m_value = 1.0  # Masa fija
  k_value = wn_value ** 2 * m_value

  # Equivalentes eléctricos
  R_value = c_value
  L_value = m_value
  C_value = 1 / k_value

  # Mostrar los valores de los parámetros calculados
  st.write("Valores calculados:")
  st.write(f"Coeficiente de amortiguamiento (c): {c_value}")
  st.write(f"Constante del resorte (k): {k_value}")
  st.write(f"Masa (m): {m_value}")
  st.write("Equivalentes eléctricos:")
  st.write(f"Resistencia (R): {R_value}")
  st.write(f"Inductancia (L): {L_value}")
  st.write(f"Capacitancia (C): {C_value}")

  # Determinar el tipo de amortiguamiento
  if zeta_value < 1:
      damping_type = "Subamortiguado (Oscilatorio)"
  elif zeta_value == 1:
      damping_type = "Amortiguamiento crítico"
  elif 1 < zeta_value < 2:
      damping_type = "Sobreamortiguado"
  else:
      damping_type = "Inestable"

  st.write(f"El sistema es: {damping_type}")

  # Función de transferencia del sistema en función de wn y zeta
  num = [wn_value ** 2]  # Numerador
  den = [1, 2 * zeta_value * wn_value, wn_value ** 2]  # Denominador
  H_s = ctl.TransferFunction(num, den)

  # Mostrar la función de transferencia
  st.write("La función de transferencia del sistema es:")
  st.latex(r"H(s) = \frac{\omega_n^2}{s^2 + 2 \zeta \omega_n s + \omega_n^2}")

  # Gráficos de simulación
  st.subheader("Gráficas de Simulación")

  # Paso 1: Gráfica de polos y ceros
  st.write("Polos y ceros:")
  plt.figure()
  ctl.pzmap(H_s, plot=True, grid=True)
  st.pyplot(plt)

  # Paso 2: Diagrama de Bode
  st.write("Diagrama de Bode:")
  plt.figure()
  mag, phase, omega = ctl.bode(H_s, dB=True, Hz=False, deg=True, plot=True)
  st.pyplot(plt)

  # Paso 3: Respuesta al impulso
  st.write("Respuesta al impulso:")
  plt.figure()
  T, yout = ctl.impulse_response(H_s)
  plt.plot(T, yout)
  plt.title('Respuesta al Impulso')
  plt.xlabel('Tiempo [s]')
  plt.ylabel('Amplitud')
  plt.grid(True)
  st.pyplot(plt)

  # Paso 4: Respuesta al escalón
  st.write("Respuesta al escalón:")
  plt.figure()
  T, yout = ctl.step_response(H_s)
  plt.plot(T, yout)
  plt.title('Respuesta al Escalón')
  plt.xlabel('Tiempo [s]')
  plt.ylabel('Amplitud')
  plt.grid(True)
  st.pyplot(plt)

#######################################################################################
elif punto == "Punto 2: Simulación Canción YouTube":
  st.write("### Simulación Canción YouTube")
  # Función para descargar audio de YouTube en formato MP3
  def download_ytvid_as_mp3(video_url, name):
      filename = f"{name}.mp3"
      options = {
          'format': 'bestaudio/best',
          'keepvideo': False,
          'outtmpl': filename,
      }
      with yt_dlp.YoutubeDL(options) as ydl:
          ydl.download([video_url])
      return filename

  # Función para convertir MP3 a WAV usando ffmpeg
  def convert_mp3_to_wav(mp3_filename, wav_filename):
      subprocess.call(['ffmpeg', '-y', '-i', mp3_filename, wav_filename])

  # Función para diseñar un filtro pasabajas
  def butter_lowpass(cutoff, fs, order=5):
      nyq = 0.5 * fs
      normal_cutoff = cutoff / nyq
      b, a = butter(order, normal_cutoff, btype='low', analog=False)
      return b, a

  def lowpass_filter(data, cutoff, fs, order=5):
      b, a = butter_lowpass(cutoff, fs, order=order)
      y = lfilter(b, a, data)
      return y, b, a

  # Función para graficar polos y ceros
  def plot_poles_zeros(b, a):
      z, p, k = tf2zpk(b, a)
      plt.figure()
      plt.scatter(np.real(z), np.imag(z), marker='o', label='Ceros', color='blue')
      plt.scatter(np.real(p), np.imag(p), marker='x', label='Polos', color='red')
      plt.axhline(0, color='black', lw=1, ls='--')
      plt.axvline(0, color='black', lw=1, ls='--')
      plt.title('Polos y Ceros')
      plt.xlabel('Parte Real')
      plt.ylabel('Parte Imaginaria')
      plt.grid()
      plt.legend()
      st.pyplot(plt)

  # Función para graficar el diagrama de Bode
  def plot_bode(b, a, fs):
      w, h = freqz(b, a, worN=8000)
      plt.figure(figsize=(10, 8))

      # Diagrama de Magnitud
      plt.subplot(2, 1, 1)
      plt.plot(0.5 * fs * w / np.pi, 20 * np.log10(np.abs(h)), 'b')
      plt.title('Diagrama de Bode')
      plt.xlabel('Frecuencia [Hz]')
      plt.ylabel('Magnitud [dB]')
      plt.grid()
      plt.ylim(-100, 10)  # Ajuste de límites en dB

      # Diagrama de Fase
      plt.subplot(2, 1, 2)
      plt.plot(0.5 * fs * w / np.pi, np.angle(h), 'b')
      plt.xlabel('Frecuencia [Hz]')
      plt.ylabel('Fase [radianes]')
      plt.grid()

      plt.tight_layout()  # Asegura que los subgráficos no se superpongan
      st.pyplot(plt)

  # Interfaz de Streamlit
  st.title("Simulación de Modulación y Demodulación de Audio desde YouTube")

  # Entrada para el enlace de YouTube
  video_url = st.text_input("Ingresa el enlace de una canción en YouTube:")

  # Parámetros ajustables con sliders
  start_time = st.slider("Tiempo de inicio del segmento (segundos)", 0, 60, 35)
  cutoff_frequency = st.slider("Frecuencia de corte del filtro (Hz)", 100, 5000, 300)

  # Procesamiento del archivo de audio si se ha ingresado un enlace
  if video_url:
      try:
          name = "audio_downloaded"
          mp3_filename = download_ytvid_as_mp3(video_url, name)
          wav_filename = f"{name}.wav"
          convert_mp3_to_wav(mp3_filename, wav_filename)

          # Leer el archivo WAV
          x, fm = sf.read(wav_filename)
          x_mono = np.mean(x, axis=1) if x.ndim > 1 else x
          ns = len(x_mono) / fm

          # Segmento seleccionado por el usuario
          end_time = start_time + 5
          start_index = int(start_time * fm)
          end_index = int(end_time * fm)

          x_mono_segment = x_mono[start_index:end_index]
          t_segment = np.arange(start_time, end_time, 1 / fm)

          # Reproducir el segmento seleccionado
          st.write(f"Reproduciendo segmento de {start_time} a {end_time} segundos.")
          with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_wav:
              sf.write(temp_wav.name, x_mono_segment, fm)
              st.audio(temp_wav.name)

          # Generar la señal portadora
          Ac = max(abs(x_mono_segment)) / 0.8  # Amplitud de la portadora
          c_segment = Ac * np.sin(2 * np.pi * 10000 * t_segment)  # Señal portadora

          # Señal modulada AM
          y = (1 + x_mono_segment / Ac) * c_segment

          # Graficar señal portadora y mensaje
          st.write("### Señal de Portadora y Mensaje")
          fig, ax = plt.subplots()
          ax.plot(t_segment, c_segment, label='Portadora', alpha=0.7)
          ax.plot(t_segment, x_mono_segment, label='Mensaje', alpha=0.7)
          ax.set_xlabel('Tiempo (s)')
          ax.set_ylabel('Amplitud')
          ax.legend()
          st.pyplot(fig)

          # Reproducción de la señal mensaje
          with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_msg:
              sf.write(temp_msg.name, x_mono_segment, fm)
              st.audio(temp_msg.name)

          # Graficar la señal modulada
          st.write("### Señal Modulada en AM")
          fig, ax = plt.subplots()
          ax.plot(t_segment, y, label='Modulada')
          ax.set_xlabel('Tiempo (s)')
          ax.set_ylabel('Amplitud')
          ax.legend()
          st.pyplot(fig)

          # Reproducción de la señal modulada
          with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_mod:
              sf.write(temp_mod.name, y, fm)
              st.audio(temp_mod.name)

          # Demodulación usando envolvente
          demodulated = np.abs(y)

          # Filtrado de la señal demodulada
          filtered_signal, b, a = lowpass_filter(demodulated, cutoff_frequency, fm)

          # Graficar la señal demodulada y filtrada
          st.write("### Señal Demodulada y Filtrada")
          fig, ax = plt.subplots()
          ax.plot(t_segment, filtered_signal[:len(t_segment)], label='Demodulada Filtrada')
          ax.set_xlabel('Tiempo (s)')
          ax.set_ylabel('Amplitud')
          ax.legend()
          st.pyplot(fig)

          # Reproducción de la señal demodulada filtrada
          with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_filt:
              sf.write(temp_filt.name, filtered_signal[:len(t_segment)], fm)
              st.audio(temp_filt.name)

          # Espectros de las señales
          st.write("### Espectros de las Señales")
          Xfc = np.fft.rfft(c_segment)
          Xfm = np.fft.rfft(x_mono_segment)
          Xfy = np.fft.rfft(y)
          Xfd = np.fft.rfft(filtered_signal[:len(t_segment)])
          vfre = np.fft.rfftfreq(len(c_segment), 1 / fm)

          fig, ax = plt.subplots()
          ax.plot(vfre, np.abs(Xfy), label='Modulada', alpha=0.7)
          ax.plot(vfre, np.abs(Xfm), label='Mensaje', alpha=0.7)
          ax.plot(vfre, np.abs(Xfc), label='Portadora', alpha=0.7)
          ax.plot(vfre, np.abs(Xfd), label='Filtrada', alpha=0.7)
          ax.set_xlabel('Frecuencia [Hz]')
          ax.set_ylabel('|X(f)|')
          ax.set_ylim([min(np.abs(Xfm)), 1.1 * max(np.abs(Xfm))])
          ax.legend()
          ax.grid(True)
          st.pyplot(fig)

          # Graficar polos y ceros
          plot_poles_zeros(b, a)

          # Graficar diagrama de Bode
          plot_bode(b, a, fm)

      except Exception as e:
          st.error(f"Error al procesar el audio: {e}")

Overwriting app.py


In [6]:
from pyngrok import ngrok

# Token personal de ngrok
token = "2lLFiUsTGsQClQyxWz9mY9rLIup_7vUusFxPVHjfdMEa1CryT"
ngrok.set_auth_token(token)

# Ejecutar Streamlit en el puerto 5011
!nohup streamlit run app.py --server.port 5011 &

# Iniciar el túnel Ngrok y exponer Streamlit en una URL.
public_url = ngrok.connect(5011)
print(f"Aplicación aquí: {public_url}")

nohup: appending output to 'nohup.out'
Aplicación aquí: NgrokTunnel: "https://2cc6-35-238-32-106.ngrok-free.app" -> "http://localhost:5011"
